# BLAST

<!-- LTeX: language=pt -->

<div align='justify'>
Em Bioinformática, a necessidade de repetir comparações de pares um grande número de vezes altera os requisitos do problema e exige soluções mais eficientes. Surgiu então o problema de encontrar sequências semelhantes a uma sequência alvo.

Neste notebook, discutimos os algoritmos heurísticos e as ferramentas existentes para esta tarefa e procedemos à
implementação de uma versão simplificada do algoritmo para comparação de sequências biológicas mais popular, o BLAST.

No último capítulo (ou última aula), discutimos como conceber e implementar algoritmos eficientes para o alinhamento de sequências por pares, que podem ser utilizados para comparar pares de sequências, a sua semelhança. Discutimos diferentes parâmetros de funções de pontuação que podem ser usados para medir a semelhança entre estas sequências.

Em muitas situações de investigação biológica, deparamo-nos com sequências de DNA ou de proteínas (por exemplo
provenientes de projectos de sequenciação de DNA) para as quais não dispomos de qualquer informação. Um pedido comum para um bioinformático é fornecer alguma anotação a esta sequência, ou seja, fornecer hipóteses para as funções da sequência.

Posto isto, a tentativa de inferir a função a partir de sequências semelhantes é uma forma de abordar esta tarefa, em que tentamos
identificar sequências que sejam semelhantes à sequência alvo (de consulta) e que já estejam anotadas com uma determinada função. 

O algoritmo BLAST é atualmente o mais utilizado para efetuar pesquisas em bases de dados de sequências de DNA ou de proteínas. O objetivo é procurar bons alinhamentos entre uma sequência de consulta e as sequências de uma base de dados definida. A ideia básica dos algoritmos é utilizar "palavras" curtas (ou seja, sub-sequências) e procurar correspondências de elevada semelhança (sem lacunas) destas palavras na na consulta e nas sequências da base de dados. Estas correspondências formarão uma base que pode ser
que pode ser alargada, em ambas as direcções, para obter alinhamentos de alta qualidade de maior dimensão

### **(*Implementação do Algoritmo Blast:*)**

Temos uma string (a query), que é a sequência que queremos procurar dentro de uma sequência alvo (seq); e um parâmetro do algoritmo, W (tamanho), que nos indica as sub-sequências de comprimento w que ocorrem na sequência de consulta.

In [31]:
#Exemplo prático
query="AATATAT"
seq="AATATGTTATATAATAATATTT"
w=3 

In [ ]:
#Exemplo para testar o Blast
query = input()
seq = input()
w = int(input())

Para desenvolver o algoritmo Blast, é necessário desenvolver quatro funções: 
- query_map() 
- hits(), socorrida de funções auxiliares ao seu funcionamento -> procura() e criar_tuplos()
- extend_hit()
- best_hit()

**query_map()**

Em primeiro lugar, constrói-se a função query_map(), que identificará todas as palavras de tamanho tamanho w (um parâmetro) que nela ocorrem, guardando para cada palavra as posições onde ela ocorre. É muito útil se for necessário encontrar repetidamente ocorrências dessas palavras na consulta. A estrutura de dados usada para guardar estes resultados será um dicionário Python, uma vez que permite um acesso muito eficiente eficiente dos valores associados às suas chaves. O dicionário devolvido tem como chaves as palavras da consulta e como valores as listas de posições onde estas ocorrem como valores.

**hits()**

De seguida, criamos a função hits, que    

In [60]:
def parse_query(query : str) -> str:
"""
  Esta função recebe uma sequência (query) e realiza as seguintes operações:
    1. Remove espaços em branco.
    2. Converte a sequência para maiúsculas.
    3. Verifica se a sequência contém apenas as letras permitidas (A, C, G, T).

  Parameters
  ----------
  query : str
    a sequência que será processada.

  Returns
  -------
  str
    a sequência processada, após a remoção de espaços, conversão para maiúsculas e validação das letras permitidas.

  Raises
  ------
  ValueError
    se a sequência for vazia ou conter caracteres não permitidos.
"""

  query = query.replace(" ", "").upper()

  if query == "": 
      raise ValueError("A sequência não pode ser vazia")
    
  letras_permitidas = set(['A', 'C', 'G', 'T'])

  if set(query) <= letras_permitidas:
      return query

  else:
      raise ValueError("A sequência deve conter apenas as letras A, C, G, T")

def query_map(query : str, w: int) -> dict[str, list[int]]:
  """
  Esta função recebe a sequência que queremos procurar dentro de uma sequência alvo (query) devolve um dicionário que
  tem como chaves as palavras da consulta e como valores as listas de posições onde estas ocorrem como valores.

  Parameters
  ----------
  query : str
    a string que procuramos.

  w : int
    o tamanho da janela.

  Returns
  -------
  dict : str, list[int]
    Devolve um dicionário:
      cuja chave são todas as strings de tamanho w da query e;
      cujo valor é uma lista com todos os offsets onde aparecem.

  Raises
  ------
  AssertionError
    se a função receber uma sequência vazia.
    se a função receber um valor negativo para w.
    se a função receber um valor não string para a sequência.
    se a função receber uma sequência com letras inválidas.
    se a função não dá o resultado esperado para esses parâmetros.
  """
  if not isinstance(query, str):
    raise TypeError("A sequência deve ser uma string")
  
  if not isinstance(w, int) or w <= 0:
    raise ValueError("O tamanho da janela deve ser um inteiro positivo")

  query = parse_query(query)

  res={}
  for I in range(len(query)):
    sub_string=query[I:I+w]

    if sub_string not in res and len(sub_string)==w:
      res[sub_string]=[I]
    elif sub_string in res and len(sub_string)==w:
      res[sub_string].append(I)
  return res

qm = query_map(query, w)

def procura(codao : str, seq : str) -> list[int]:
  """
  Função auxiliar da função hits.
  Esta função procura uma subsequência específica (codao) em uma sequência dada (seq) e
  retorna uma lista contendo os offsets onde a subsequência é encontrada.

  Parameters
  ----------
  codao : str
    a subsequência que estamos procurando na sequência.

  seq : str
    a sequência na qual estamos procurando a subsequência.

  Returns
  -------
  list[int]
    uma lista contendo os offsets (índices) onde a subsequência é encontrada na sequência.
  """
  tm = len(codao)
  indices = []
  for i in range(len(seq)):
      sub_string = seq[i: i + tm]
      if codao == sub_string:
          indices.append(i)
  return indices


def criar_tuplos(indices : list[int], offset : int) -> list[tuple[int, int]]: 
  """
  Função auxiliar da função hits.
  Esta função cria tuplos contendo o offset fornecido e os elementos da lista de índices.

  Parameters
  ----------
  indices : list[int]
    lista contendo os índices para os quais os tuplos serão criados.

  offset : int
    o valor do offset que será incluído nos tuplos.

  Returns
  -------
  list[tuple[int, int]]
    Lista de tuplos contendo o offset e cada elemento da lista de índices.
  """
  tuplos = []

  for l in indices:
      tuplos.append((offset, l))
  return tuplos


def hits(qm : dict[str, list[int]], seq : str) -> list[tuple[int, int]]:
  """
  Esta função recebe ... devolve...

  Parameters
  ----------
  qm : dict[str, list[int]]
    o que é devolvido ao invocar a função query_map.

  seq : str
    a sequência alvo.

  Returns
  -------
  list : tuple[int, int]
    Devolve uma lista em que cada elemento é um tuplo com dois valores:
      1. O Offset na query;
      2. O offset na seq.

  Raises
  ------
  AssertionError
    se a função receber uma sequência vazia.
    se a função receber um valor não string para a sequência.
    se a função receber uma sequência com letras inválidas.
    se a função não dá o resultado esperado para esses parâmetros.
  """
  if not isinstance(seq, str):
    raise TypeError("A sequência deve ser uma string")
  hits_list = [] 

  seq = parse_query(seq)

  for query, offsets in qm.items():
      # seq <- indices query
      res = procura(query, seq)
      # [0 , 12, 15] 0 -> 0,0 0,12 0,15
      for offset in offsets:
          hits_list += criar_tuplos(res, offset) 

  return hits_list

res = hits(qm, seq)

def extend_hit(query : str, seq : str, hit : tuple[int, int], w: int) -> tuple[int, int, int, int]:
  """
  Esta função recebe ... devolve...
      
  Parameters
  ----------
  query : str
    a string que procuramos.

  seq : str
    a sequência alvo.

  hit : tuple[int, int]
    um dos elementos devolvidos pela invocação da função hits.

  w : int
    o tamanho da janela.

  Returns
  -------
  tuple : [int, int, int, int]
    Devolve um tuplo com:
        1. O offset inicial na query;
        2. O offset inicial na seq;
        3. O tamanho do resultado;
        4. O nº de matches corretos.

  Raises
  ------
  AssertionError
    se a função receber uma sequência vazia.
    se a função receber uma query vazia.
    se a função receber um valor negativo para w.
    se a função receber um valor não string para a sequência.
    se a função receber um valor não string para a query.
    se a função receber uma sequência com letras inválidas.
    se a função receber uma query com letras inválidas.
    se a função não dá o resultado esperado para esses parâmetros.
  """
  if not isinstance(query, str):
    raise TypeError("A sequência deve ser uma string")
  if not isinstance(seq, str):
    raise TypeError("A sequência deve ser uma string")
  if not isinstance(w, int) or w <= 0:
    raise ValueError("O tamanho da janela deve ser um inteiro positivo")
  
  query = parse_query(query)
  seq = parse_query(seq)

  num_matches = w
  tam_extensao = w 
  i_min_query = hit[0]
  i_min_seq = hit[1]
  i_max_query = hit[0] + w - 1
  i_max_seq = hit[1] + w - 1
    
  while num_matches >= tam_extensao / 2 and ((i_min_seq > 0 and i_min_query > 0) or (i_max_seq < len(seq) - 1 and i_max_query < len(query) - 1)):
      # esquerda 
      if i_min_query>0 and i_min_seq>0:
          i_min_query -= 1
          i_min_seq -= 1 
          tam_extensao += 1
          if query[i_min_query] == seq[i_min_seq]:
              num_matches += 1
      # direita 
      if i_max_query < len(query) - 1 and i_max_seq < len(seq) - 1:
          i_max_query += 1
          i_max_seq += 1 
          tam_extensao += 1
          if query[i_max_query] == seq[i_max_seq]:
              num_matches += 1
            
  return (i_min_query, i_min_seq, i_max_query - i_min_query + 1, num_matches)


def best_hit(query : str, seq : str, w: int) -> tuple[int, int, int, int]:
  """
  Esta função recebe ... devolve...

  Parameters
  ----------
  query : str
    a string que procuramos.

  seq : str
    a sequência alvo.

  w : int
    o tamanho da janela.

  Returns
  -------
  tuple : [int, int, int, int]
    Devolve um tuplo com:
      1. O offset inicial na query;
      2. O offset inicial na seq;
      3. O tamanho do resultado;
      4. O nº de matches corretos.

  Raises
  ------
  AssertionError
    se a função receber uma sequência vazia.
    se a função receber uma query vazia.
    se a função receber um valor negativo para w.
    se a função receber um valor não string para a sequência.
    se a função receber um valor não string para a query.
    se a função receber uma sequência com letras inválidas.
    se a função receber uma query com letras inválidas.
    se a função não dá o resultado esperado para esses parâmetros.
  """
  if not isinstance(query, str):
    raise TypeError("A sequência deve ser uma string")
  if not isinstance(seq, str):
    raise TypeError("A sequência deve ser uma string")
  if not isinstance(w, int) or w <= 0:
    raise ValueError("O tamanho da janela deve ser um inteiro positivo")
  
  query = parse_query(query)
  seq = parse_query(seq)

  qm = query_map(query, w)
  hits_list = hits(qm, seq)
  
  best_hit_result = None
  best_score = 0

  for hit in hits_list:
      extended_hit = extend_hit(query, seq, hit, w)
      score = extended_hit[3]

      if score > best_score or (score == best_score and extended_hit[2] < best_hit_result[2]):
        best_hit_result = extended_hit
        best_score = score

  return best_hit_result

In [20]:
best_hit(query, seq, w)

(0, 0, 7, 6)

### Testes de unidade

In [25]:
import unittest

#Para a função query_map
class TestBlast(unittest.TestCase):
    def test_query_map_vazia(self):
        # Testa se a função lança um erro ao receber uma sequência vazia
        with self.assertRaises(ValueError):
            query_map("", 3)

    def test_query_map_w_negativo(self):
        # Testa se a função lança um erro ao receber um valor negativo para w
        with self.assertRaises(ValueError):
            query_map("ACT", -3)

    def test_query_map_w_number(self):
        # Testa se a função lança um erro ao receber um valor não string para a sequência
        with self.assertRaises(TypeError):
            query_map(1, "3")

    def test_query_map_invalida(self):
        # Testa se a função lança um erro ao receber uma sequência com letras inválidas
        with self.assertRaises(ValueError):
            query_map("epQINM", 3)
            
    def test_query_map_output(self):
        # Testa se a função dá o resultado esperado para esses parâmetros
        output = {'ATG': [0], 'TGC': [1], 'GCG': [2], 'CGG': [3], 'GGT': [4]}
        self.assertEqual(query_map("ATGCGGT", 3), output)

suite = unittest.TestLoader().loadTestsFromTestCase(TestBlast)
unittest.TextTestRunner(verbosity=3).run(suite)

test_query_map_invalida (__main__.TestBlast.test_query_map_invalida) ... ok
test_query_map_output (__main__.TestBlast.test_query_map_output) ... ok
test_query_map_vazia (__main__.TestBlast.test_query_map_vazia) ... ok
test_query_map_w_negativo (__main__.TestBlast.test_query_map_w_negativo) ... ok
test_query_map_w_number (__main__.TestBlast.test_query_map_w_number) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.025s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

In [39]:
#Para a função hits
class TestBlast(unittest.TestCase):
    def test_hit_vazia(self):
        # Testa se a função lança um erro ao receber uma sequência vazia
        with self.assertRaises(ValueError):
            hits(qm, "")

    def test_hit_w_number(self):
        # Testa se a função lança um erro ao receber um valor não string para a sequência
        with self.assertRaises(TypeError):
            hits(qm, 34)

    def test_hit_invalida(self):
        # Testa se a função lança um erro ao receber uma sequência com letras inválidas
        with self.assertRaises(ValueError):
            hits(qm, "Efehsw")
            
    def test_hit_output(self):
        # Testa se a função dá o resultado esperado para esses parâmetros
        q = query_map("AATATAT", 3)
        output = [(0, 0), (0, 12), (0, 15), (1, 1), (1, 8), (1, 10), (1, 13), (1, 16), (3, 1), (3, 8), (3, 10), (3, 13), (3, 16), (2, 2), (2, 7), (2, 9), (2, 17), (4, 2), (4, 7), (4, 9), (4, 17)]
        self.assertEqual(hits(q, "AATATGTTATATAATAATATTT"), output)

suite = unittest.TestLoader().loadTestsFromTestCase(TestBlast)
unittest.TextTestRunner(verbosity=3).run(suite)

test_hit_invalida (__main__.TestBlast.test_hit_invalida) ... ok
test_hit_output (__main__.TestBlast.test_hit_output) ... ok
test_hit_vazia (__main__.TestBlast.test_hit_vazia) ... ok
test_hit_w_number (__main__.TestBlast.test_hit_w_number) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.017s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

In [61]:
#Para a função extend_hit
class TestBlast(unittest.TestCase):
    def test_extend_hit_seq_vazia(self):
        # Testa se a função lança um erro ao receber uma sequência vazia
        with self.assertRaises(ValueError):
            extend_hit("AATATAT", "", (0, 12), 3)

    def test_extend_hit_query_vazia(self):
        # Testa se a função lança um erro ao receber uma query vazia
        with self.assertRaises(ValueError):
            extend_hit("", "AATATGTTATATAATAATATTT", (0, 12), 3)

    def test_extend_hit_w_negativo(self):
        # Testa se a função lança um erro ao receber um valor negativo para w
        with self.assertRaises(ValueError):
            extend_hit("AATATAT", "AATATGTTATATAATAATATTT", (0, 12), -9)

    def test_extend_hit_w_number_seq(self):
        # Testa se a função lança um erro ao receber um valor não string para a sequência
        with self.assertRaises(TypeError):
            extend_hit("AATATAT", 33, (0, 12), 3)

    def test_extend_hit_w_number_query(self):
        # Testa se a função lança um erro ao receber um valor não string para a query
        with self.assertRaises(TypeError):
            extend_hit(33, "AATATGTTATATAATAATATTT", (0, 12), 3)

    def test_extend_hit_seq_invalida(self):
        # Testa se a função lança um erro ao receber uma sequência com letras inválidas
        with self.assertRaises(ValueError):
            extend_hit("AATATAT", "SWiopgF", (0, 12), 3)

    def test_extend_hit_query_invalida(self):
        # Testa se a função lança um erro ao receber uma query com letras inválidas
        with self.assertRaises(ValueError):
            extend_hit("qwiofgr", "AATATGTTATATAATAATATTT", (0, 12), 3)
            
    def test_extend_hit_output(self):
        # Testa se a função dá o resultado esperado para esses parâmetros
        output = (0, 12, 7, 4)
        self.assertEqual(extend_hit("AATATAT", "AATATGTTATATAATAATATTT", (0, 12), 3), output)

suite = unittest.TestLoader().loadTestsFromTestCase(TestBlast)
unittest.TextTestRunner(verbosity=3).run(suite)

test_extend_hit_output (__main__.TestBlast.test_extend_hit_output) ... ok
test_extend_hit_query_invalida (__main__.TestBlast.test_extend_hit_query_invalida) ... ok
test_extend_hit_query_vazia (__main__.TestBlast.test_extend_hit_query_vazia) ... ok
test_extend_hit_seq_invalida (__main__.TestBlast.test_extend_hit_seq_invalida) ... ok
test_extend_hit_seq_vazia (__main__.TestBlast.test_extend_hit_seq_vazia) ... ok
test_extend_hit_w_negativo (__main__.TestBlast.test_extend_hit_w_negativo) ... ok
test_extend_hit_w_number_query (__main__.TestBlast.test_extend_hit_w_number_query) ... ok
test_extend_hit_w_number_seq (__main__.TestBlast.test_extend_hit_w_number_seq) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.069s

OK


<unittest.runner.TextTestResult run=8 errors=0 failures=0>

In [58]:
#Para a função best_hit
class TestBlast(unittest.TestCase):
    def test_best_hit_seq_vazia(self):
        # Testa se a função lança um erro ao receber uma sequência vazia
        with self.assertRaises(ValueError):
            best_hit("AATATAT", "", 3)

    def test_best_hit_query_vazia(self):
        # Testa se a função lança um erro ao receber uma query vazia
        with self.assertRaises(ValueError):
            best_hit("", "AATATGTTATATAATAATATTT", 3)

    def test_best_hit_w_negativo(self):
        # Testa se a função lança um erro ao receber um valor negativo para w
        with self.assertRaises(ValueError):
            best_hit("AATATAT", "AATATGTTATATAATAATATTT", -3)

    def test_best_hit_w_number_seq(self):
        # Testa se a função lança um erro ao receber um valor não string para a sequência
        with self.assertRaises(TypeError):
            best_hit("AATATAT", 33, 3)

    def test_best_hit_w_number_query(self):
        # Testa se a função lança um erro ao receber um valor não string para a query
        with self.assertRaises(TypeError):
            best_hit(33, "AATATGTTATATAATAATATTT", 3)

    def test_best_hit_seq_invalida(self):
        # Testa se a função lança um erro ao receber uma sequência com letras inválidas
        with self.assertRaises(ValueError):
            best_hit("AATATAT", "SWiopgF", 3)

    def test_best_hit_query_invalida(self):
        # Testa se a função lança um erro ao receber uma query com letras inválidas
        with self.assertRaises(ValueError):
            best_hit("qwiofgr", "AATATGTTATATAATAATATTT", 3)
            
    def test_best_hit_output(self):
        # Testa se a função dá o resultado esperado para esses parâmetros
        output = (0, 0, 7, 6)
        self.assertEqual(best_hit("AATATAT", "AATATGTTATATAATAATATTT", 3), output)

suite = unittest.TestLoader().loadTestsFromTestCase(TestBlast)
unittest.TextTestRunner(verbosity=3).run(suite)

test_best_hit_output (__main__.TestBlast.test_best_hit_output) ... ok
test_best_hit_query_invalida (__main__.TestBlast.test_best_hit_query_invalida) ... ok
test_best_hit_query_vazia (__main__.TestBlast.test_best_hit_query_vazia) ... ok
test_best_hit_seq_invalida (__main__.TestBlast.test_best_hit_seq_invalida) ... ok
test_best_hit_seq_vazia (__main__.TestBlast.test_best_hit_seq_vazia) ... ok
test_best_hit_w_negativo (__main__.TestBlast.test_best_hit_w_negativo) ... ok
test_best_hit_w_number_query (__main__.TestBlast.test_best_hit_w_number_query) ... ok
test_best_hit_w_number_seq (__main__.TestBlast.test_best_hit_w_number_seq) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.029s

OK


<unittest.runner.TextTestResult run=8 errors=0 failures=0>